In [61]:
import xarray as xr
from pathlib import Path
import platform
import yaml
import numpy as np
from ascat.read_native.ragged_array_ts import CellFileCollection
from tqdm import tqdm  # import tqdm for progress bar
import matplotlib.pyplot as plt
import rioxarray
from rasterio.warp import transform

## Plans

Now we want to match the Sentinel1 6km Equi7 grid to the Ascat 12.5km Fibonaccy grid. We just want to align the spatial resolution. We will not align the time dimension. We could use
- Nearest Neighbour (not super nice as we have 6km and 12.5km resolution)
- Some Interpolation Technique

When we look at the pixle size it would be recommended to use around nine 6km pixles from the rasta data for one 12.5km pixle in the point data.

sig_dc_linear_12p5_fibgrid = sig_dc_linear_6p25_fibgrid.nearest.neighbour

sig_dc_12p5_fibgrid = 10 * np.log10(sig_dc_linear_12p5_fibgrid)
sig_dc_12p5_fibgrid

### Loading Sentinel 1 data

In [80]:
sentinel_ds = xr.open_zarr("output_6km_filled.zarr")
sentinel_ds

<xarray.Dataset> Size: 6kB
Dimensions:      (time: 2, y: 21, x: 17)
Coordinates:
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 16B 2022-01-06T05:00:00 2022-01-07T17:...
  * x            (x) float64 136B 4.483e+06 4.489e+06 ... 4.576e+06 4.58e+06
  * y            (y) float64 168B 1.382e+06 1.376e+06 ... 1.264e+06 1.26e+06
Data variables:
    VH           (time, y, x) float32 3kB dask.array<chunksize=(1, 20, 16), meta=np.ndarray>
    VV           (time, y, x) float32 3kB dask.array<chunksize=(1, 20, 16), meta=np.ndarray>

In [81]:
# get bounds for ASCAT data from S1

# is this a good approach? look at step 1 to see why

sentinel_ds = sentinel_ds.rio.write_crs(sentinel_ds.spatial_ref.attrs['crs_wkt'])

#bounds_proj = sentinel_ds.rio.bounds()

sentinel_ds_latlon = sentinel_ds.rio.reproject("EPSG:4326")

latmin = float(sentinel_ds_latlon.y.min())
latmax = float(sentinel_ds_latlon.y.max())
lonmin = float(sentinel_ds_latlon.x.min())
lonmax = float(sentinel_ds_latlon.x.max())

bbox=(latmin, latmax, lonmin, lonmax)

print(f"bbox: {bbox}")

bbox: (43.92326671136945, 45.09195053816223, 6.779846242494321, 8.208237586352164)


We can see that we do not get the exact defined lat and lon we defined but a slightly bigger area. This should not be a problem in our further analysis.

### Reading ASCAT data points

In [45]:
# read yaml for reading data on Windows, Linux or the JupyterHub
paths = yaml.safe_load(Path("../paths.yml").read_text())
root = Path(paths["windows" if platform.system() == "Windows" else "linux"]).expanduser()
cell_source = root / "datasets/scat_ard/ascat_ssm_cdr_12.5km_h121"
assert cell_source.exists(), f"Path does not exist: {cell_source}"

documentation from:
https://github.com/TUW-GEO/ascat/blob/master/docs/org/reading_cells.org

In [46]:
from ascat.cell import CellGridFiles

h121_reader = CellGridFiles.from_product_id(cell_source, "H121")

ascat_ds = h121_reader.read(bbox=bbox)
ascat_ds

<xarray.Dataset> Size: 111MB
Dimensions:                            (obs: 1680202, locations: 95)
Coordinates:
    time                               (obs) datetime64[ns] 13MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
    lat                                (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
    alt                                (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
Dimensions without coordinates: obs, locations
Data variables: (12/21)
    as_des_pass                        (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    topographic_complexity             (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    subsurface_scattering_probability  (obs) float64 13MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    sat_id                             (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    row_size                           (locations) int64 760B dask.array<chunksize=(95,), meta=np.ndarray>
    location_id                        (locations) int64 760B dask.array<chunksize=(95,), meta=np.ndarray>
    location_description               (locations) <U1 380B dask.array<chunksize=(95,), meta=np.ndarray>
Attributes:
    id:                 1358.nc
    date_created:       2025-01-09 12:40:43
    featureType:        timeSeries
    grid_mapping_name:  fibgrid_12.5

In [67]:
points_ds = ascat_ds.cf_geom.to_point_array()
points_ds

<xarray.Dataset> Size: 151MB
Dimensions:                            (obs: 1680202)
Coordinates:
    time                               (obs) datetime64[ns] 13MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (obs) float32 7MB 7.321 7.321 ... 8.143
    lat                                (obs) float32 7MB 43.94 43.94 ... 45.09
    alt                                (obs) float32 7MB 9.969e+36 ... 9.969e+36
Dimensions without coordinates: obs
Data variables: (12/20)
    as_des_pass                        (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    wetland_fraction                   (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    topographic_complexity             (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    subsurface_scattering_probability  (obs) float64 13MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    sat_id                             (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    location_id                        (obs) int64 13MB 1140858 ... 1164580
    location_description               (obs) <U1 7MB '' '' '' '' ... '' '' '' ''
Attributes:
    id:                 1358.nc
    date_created:       2025-01-09 12:40:43
    featureType:        point
    grid_mapping_name:  fibgrid_12.5

In [68]:
from pyproj import Transformer
x, y = Transformer.from_crs("EPSG:4326", "EPSG:27704", always_xy=True).transform(points_ds.lon, points_ds.lat)
x

array([4503273.44545286, 4503273.44545286, 4503273.44545286, ...,
       4594099.7482635 , 4594099.7482635 , 4594099.7482635 ])

In [75]:
points_ds['e7x'] = ('obs', x)
points_ds['e7y'] = ('obs', y)

In [76]:
points_ds

<xarray.Dataset> Size: 178MB
Dimensions:                            (obs: 1680202)
Coordinates:
    time                               (obs) datetime64[ns] 13MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (obs) float32 7MB 7.321 7.321 ... 8.143
    lat                                (obs) float32 7MB 43.94 43.94 ... 45.09
    alt                                (obs) float32 7MB 9.969e+36 ... 9.969e+36
Dimensions without coordinates: obs
Data variables: (12/22)
    as_des_pass                        (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    subsurface_scattering_probability  (obs) float64 13MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    sat_id                             (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    location_id                        (obs) int64 13MB 1140858 ... 1164580
    location_description               (obs) <U1 7MB '' '' '' '' ... '' '' '' ''
    e7x                                (obs) float64 13MB 4.503e+06 ... 4.594...
    e7y                                (obs) float64 13MB 1.263e+06 ... 1.375...
Attributes:
    id:                 1358.nc
    date_created:       2025-01-09 12:40:43
    featureType:        point
    grid_mapping_name:  fibgrid_12.5

In [82]:
ascat_sentinel = sentinel_ds.sel(x=points_ds.e7x, y=points_ds.e7y, time=points_ds.time, method="nearest")
ascat_sentinel

<xarray.Dataset> Size: 74MB
Dimensions:      (obs: 1680202)
Coordinates:
    time         (obs) datetime64[ns] 13MB 2022-01-06T05:00:00 ... 2022-01-07...
    x            (obs) float64 13MB 4.501e+06 4.501e+06 ... 4.58e+06 4.58e+06
    y            (obs) float64 13MB 1.264e+06 1.264e+06 ... 1.376e+06 1.376e+06
    spatial_ref  int32 4B 0
    lon          (obs) float32 7MB 7.321 7.321 7.321 7.321 ... 8.143 8.143 8.143
    lat          (obs) float32 7MB 43.94 43.94 43.94 43.94 ... 45.09 45.09 45.09
    alt          (obs) float32 7MB 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
Dimensions without coordinates: obs
Data variables:
    VH           (obs) float32 7MB dask.array<chunksize=(320,), meta=np.ndarray>
    VV           (obs) float32 7MB dask.array<chunksize=(320,), meta=np.ndarray>

In [83]:
ascat_ds['s1-VH'] = ("obs", ascat_sentinel['VH'].data)
ascat_ds['s1-VV'] = ("obs", ascat_sentinel['VV'].data)
ascat_ds

<xarray.Dataset> Size: 151MB
Dimensions:                            (obs: 1680202, locations: 95)
Coordinates:
    time                               (obs) datetime64[ns] 13MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
    lat                                (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
    alt                                (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
Dimensions without coordinates: obs, locations
Data variables: (12/25)
    as_des_pass                        (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 2MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 7MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    location_id                        (locations) int64 760B dask.array<chunksize=(95,), meta=np.ndarray>
    location_description               (locations) <U1 380B dask.array<chunksize=(95,), meta=np.ndarray>
    e7x                                (obs) float64 13MB 4.503e+06 ... 4.594...
    e7y                                (obs) float64 13MB 1.263e+06 ... 1.375...
    s1-VH                              (obs) float32 7MB dask.array<chunksize=(320,), meta=np.ndarray>
    s1-VV                              (obs) float32 7MB dask.array<chunksize=(320,), meta=np.ndarray>
Attributes:
    id:                 1358.nc
    date_created:       2025-01-09 12:40:43
    featureType:        timeSeries
    grid_mapping_name:  fibgrid_12.5

### Mapping the Sentinel 1 data 
We want to map the data to the Fibonaccy Pointgrid to match the 12.5km resolution ASCAT data

For sampling we use Nearest Neighbour

#### Step 1
1.1 Which points are in my area and how do i get their location_id and lat lon? (maybe the read() is to unnecessary because i do not need the values)
1.2 Reproject the ASCAT points to the Sentinel 1 raster

In [47]:
ascat_points = ascat_ds[['location_id']] 
ascat_points

<xarray.Dataset> Size: 2kB
Dimensions:      (locations: 95)
Coordinates:
    lon          (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
    lat          (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
    alt          (locations) float32 380B dask.array<chunksize=(95,), meta=np.ndarray>
Dimensions without coordinates: locations
Data variables:
    location_id  (locations) int64 760B dask.array<chunksize=(95,), meta=np.ndarray>
Attributes:
    id:                 1358.nc
    date_created:       2025-01-09 12:40:43
    featureType:        timeSeries
    grid_mapping_name:  fibgrid_12.5

In [48]:
ascat_ids = ascat_ds['location_id'].values
ascat_lons = ascat_ds['lon'].values
ascat_lats = ascat_ds['lat'].values

In [59]:
sentinel_crs = sentinel_ds.spatial_ref.attrs['projected_crs_name']

if sentinel_crs == 'WGS 84 / Equi7 Europe':
    sentinel_epsg = '27704'
elif sentinel_crs == 'WGS 84 / Equi7 Africa':
    sentinel_epsg = '27701'

In [60]:
from pyproj import Transformer


transformer = Transformer.from_crs(
    "EPSG:4326", 
    sentinel_epsg, 
    always_xy=True  # Ensures lon=x, lat=y order
)

ascat_x, ascat_y = transformer.transform(ascat_lons, ascat_lats)
ascat_x, ascat_y

(array([4503273.44545286, 4557956.83737502, 4470131.3926819 ,
        4524735.60191594, 4491578.42299134, 4546192.25233846,
        4513020.43240459, 4567643.31739221, 4479913.07504325,
        4534457.16413084, 4501334.83553053, 4555888.23701355,
        4468277.50171223, 4522751.45901025, 4489678.91143173,
        4544162.65135617, 4511075.32486668, 4565568.06947776,
        4478052.7602111 , 4532466.44768373, 4499428.8618383 ,
        4553851.93561444, 4520799.85075549, 4575231.51067228,
        4487812.17002769, 4542165.26946478, 4509162.88596045,
        4563524.95285127, 4476225.34950055, 4530508.17120402,
        4497555.69080113, 4551847.94943482, 4518880.74097198,
        4573181.74926158, 4485978.27255495, 4540200.4252854 ,
        4507283.07887175, 4561514.27029513, 4474430.91589638,
        4528582.56773927, 4582822.15906835, 4495715.2849685 ,
        4549876.44500523, 4516994.47690043, 4571164.3296964 ,
        4484177.55252489, 4538268.19733401, 4505436.34466273,
        

#### Step 2

Get nearest neighbours for the ascat data

In [ ]:
import numpy as np
from scipy.spatial import KDTree
import xarray as xr

ascat_points = np.column_stack((ascat_x, ascat_y))
tree = KDTree(ascat_points)


'27704'

AttributeError: 'ellipsis' object has no attribute 'spatial_ref'

#### Step 3
Attach samples values to the ascat points

### Old stuff


In [ ]:
# old approach crs is not the same i think

ascat_crs = "EPSG:4326"                        # ASCAT in geographic coordinates
sentinel_crs = sentinel_ds.rio.crs.to_string() # Sentinel-1 in projected CRS (e.g., EPSG:3035)

ascat_x, ascat_y = transform(
    src_crs=ascat_crs,
    dst_crs=sentinel_crs,
    xs=ascat_ds["lon"].values,
    ys=ascat_ds["lat"].values,
)


# check transtormation of ascat points on sentinel data

vv_img = sentinel_ds["VV"].isel(time=0)

fig, ax = plt.subplots(figsize=(8, 6))
vv_img.plot(ax=ax, cmap="viridis", robust=True)
ax.scatter(ascat_x, ascat_y, s=10, c='red', label="ASCAT points", alpha=0.8)
ax.set_xlim(0.99*min(np.min(ascat_x), sentinel_ds.x.min().item()), 
            1.01*max(np.max(ascat_x), sentinel_ds.x.max().item()))
ax.set_ylim(0.99*min(np.min(ascat_y), sentinel_ds.y.min().item()), 
            1.01*max(np.max(ascat_y), sentinel_ds.y.max().item()))
ax.set_title("Sentinel-1 VV with ASCAT Points")
ax.legend()
plt.show()